In [1]:
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
import matplotlib.pyplot as plt

# Optimization module in scipy
import scipy.optimize as opt

# Module to load MATLAB .mat datafile format (Input and output module of scipy)
from scipy.io import loadmat

# Python Imaging Library (PIL)
from PIL import Image

# tells matplotlib to embed plots within the notebook
%matplotlib inline

In [2]:
Data_Dir = 'DATA'
data = loadmat(os.path.join(Data_Dir, 'mnist-digit.mat'))

In [3]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

In [4]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [5]:
data['y'][data['y']==10]=0

In [6]:
np.unique(data['y'])

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [7]:
# def displayData(X,y):
#     """
#     Displays the data from X
#     """
#     # Create figure
#     fig, ax = plt.subplots(nrows=10, ncols=10, sharex=True, sharey=True, figsize=(10, 12))
#     for r in range(10):
#         for c in range(10):
#             res = random.sample(range(1, 5000), 1)
#             ax[r, c].matshow(X[res][0].reshape((20, 20)).T, cmap='binary')
#             ax[r,c].title.set_text(y[res][0])
#             plt.xticks(np.array([]))
#             plt.yticks(np.array([]))
#     plt.show();

In [8]:
# displayData(data['X'],data['y'])

#### Regularized logistic regression

Now add regularization to the cost function. For regularized logistic regression, the cost function is defined as

$$ J(w) = \frac{1}{m} \sum_{i=1}^m \left[ -y^{(i)} \log \left(h_w\left(x^{(i)} \right)\right) - \left( 1 - y^{(i)} \right) \log\left(1 - h_w \left(x^{(i)} \right) \right) \right] + \frac{\lambda}{2m} \sum_{j=1}^n w_j^2 $$

Note that $w_0$ should not be regularized as it is used as bias term.
Correspondingly, the partial derivative of regularized logistic regression cost for $w_j$ is defined as

$$
\begin{align*}
& \frac{\partial J(w)}{\partial w_0} = \frac{1}{m} \sum_{i=1}^m \left( h_w\left( x^{(i)} \right) - y^{(i)} \right) x_j^{(i)}  & \text{for } j = 0 \\
& \frac{\partial J(w)}{\partial w_j} = \left( \frac{1}{m} \sum_{i=1}^m \left( h_w\left( x^{(i)} \right) - y^{(i)} \right) x_j^{(i)} \right) + \frac{\lambda}{m} w_j & \text{for } j  \ge 1
\end{align*}
$$


In [9]:
def sigmoid(z):
    """
    Computes the sigmoid of z
    """
    return 1/(1+np.exp(-z))

In [10]:
X =  data['X']

In [11]:
X.shape

(5000, 400)

In [12]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
def lrCostFunction(w,X,y,lambda_):
    m,n = X.shape
    if y.dtype == bool:
        y = y.astype(int)
    

In [14]:
def lrCostFunction(w, X, y, lambda_):
    
    m = y.size
    
    if y.dtype == bool:
        y = y.astype(int)
        
    grad = np.zeros(w.shape)
    
    h = sigmoid(np.dot(X,w))
    print('Shape',(h-y).shape)
    
    J = (1/m)*np.sum(-y*np.log(h + 1e-10)-(1-y)*np.log(1-h + 1e-10)) + (lambda_/(2*m))*np.sum(w[1:]**2)
    
    grad_w0 =(1/m)*np.sum(np.dot(X[:,0],(h-y)))
    grad_w = (1/m)*(np.dot(X[:,1:],(h-y))) + (lambda_/m)*w[1:]
    grad[0]=grad_w0
    grad[1:]=grad_w
    return J, grad

In [15]:
y = data['y']
y.shape

(5000, 1)

In [16]:
y.squeeze().shape

(5000,)

In [17]:
# def lrCostFunction(w, X, y, lambda_):
    
#     m = y.size
    
#     if y.dtype == bool:
#         y = y.astype(int)
        
#     # y = y.squeeze()
#     h = sigmoid(np.dot(X,w))
    
#     J = (1/m)*np.sum(-y*np.log(h + 1e-10)-(1-y)*np.log(1-h + 1e-10)) + (lambda_/(2*m))*np.sum(w[1:]**2)
#     # print(X.T.shape)
#     # print(y.shape)
#     # print((h-y).shape)
#     # print(w.shape)
#     grad_ = (1/m)*np.dot(X.T,(h-y).squeeze()) + (lambda_/m)*w
#     grad_w0 =(1/m)*np.dot(X.T,(h-y))[0]
#     grad_[0]=grad_w0
#     return J, grad_

In [18]:
y = data['y']
y = np.where(y==1,1,0)
y = y.squeeze()
X = data['X']
m , n = X.shape
X = np.c_[np.ones(m), X]
w = np.zeros((X.shape[1]))
np.unique(y),w.shape

(array([0, 1]), (401,))

In [19]:
lambda_=0.1

In [20]:
import scipy

In [21]:
J_history = []
epoch =1
def callbackF(w):
    """
    Callback function for scipy.optimize.minimize
    """
    global epoch
    global J_history
    J, grad = lrCostFunction(w, X, y, lambda_)
    J_history.append(J)
    print('Cost at epoch {}: {:f}'.format(epoch, J), end='\r')
    epoch += 1
    

In [22]:
res =scipy.optimize.minimize(lrCostFunction, w, args=(X,y,lambda_), method='CG', jac=True,callback=callbackF, options={'maxiter': 50}, tol=1e-5)

Shape (5000,)


ValueError: shapes (5000,400) and (5000,) not aligned: 400 (dim 1) != 5000 (dim 0)

In [23]:
res.x.shape

NameError: name 'res' is not defined

In [24]:
def score(X,y,w):
    m = y.size
    h = sigmoid(np.dot(X,w))
    h = np.where(h>=0.5,1,0)
    return np.sum(h==y)/m

In [183]:
w = res.x
score(X,y,w)

0.9978

In [196]:
all_w = np.zeros((10,401 ))
all_w[0] = res.x

In [199]:
all_w[0].shape

(401,)

In [198]:
all_w.shape

(10, 401)

In [226]:
def lrCostFunction(w, X, y, lambda_):
    
    m = y.size
    
    if y.dtype == bool:
        y = y.astype(int)
        
    grad = np.zeros(w.shape)
    
    h = sigmoid(np.dot(X,w))
    # print('Shape',(h-y).shape)
    
    J = (1/m)*np.sum(-y*np.log(h + 1e-10)-(1-y)*np.log(1-h + 1e-10)) + (lambda_/(2*m))*np.sum(w[1:]**2)
    
    grad_w0 =(1/m)*np.sum(np.dot(X[:,0],(h-y)))
    grad_w = (1/m)*(np.dot(X[:,1:].T,(h-y))) + (lambda_/m)*w[1:]
    grad[0]=grad_w0
    grad[1:]=grad_w
    # print(grad.shape)
    return J, grad

In [227]:
X[:,1:].T.shape

(400, 5000)

In [228]:
def oneVsAll(X, y, num_labels, lambda_):
        
    m, n = X.shape
    
    all_w = np.zeros((num_labels, n ))
    J_all = np.zeros((num_labels, 50))
    
    for c in range(num_labels):
        print('Training for class {}'.format(c), end='\r')
        y = np.where(y==c,1,0)
        y = y.squeeze()
        w = np.zeros((X.shape[1]))
        h = sigmoid(np.dot(X,w))
        res =scipy.optimize.minimize(lrCostFunction, w, args=(X,y,lambda_), method='CG', jac=True, options={'maxiter': 50}, tol=1e-5)
        all_w[c] = res.x
        # J_all[c] = J_history
    return all_w

In [229]:
y = data['y']

X = data['X']
m , n = X.shape
X = np.c_[np.ones(m), X]
print(X.shape)
w = np.zeros((X.shape[1]))
np.unique(y),w.shape

(5000, 401)


(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), (401,))

In [231]:
all_w=oneVsAll(X,y,10,0.1)

In [232]:
all_w.shape

(10, 401)

In [233]:
def predict(X,all_w):
    m = X.shape[0]
    h = sigmoid(np.dot(X,all_w.T))
    return np.argmax(h,axis=1)

In [237]:
y_pred=predict(X,all_w)

In [238]:
y.shape

(5000, 1)

In [236]:
def accuracy(y,y_pred):
    return np.sum(y==y_pred)/y.size

In [239]:
accuracy(y.squeeze(),y_pred)

0.1438